In [18]:
import logging
import random

import numpy as np
import pandas as pd

from gensim.models import doc2vec
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def read_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter="\t")
    x_train, x_test, y_train, y_test = train_test_split(dataset.review, dataset.sentiment, random_state=0, test_size=0.1)
    x_train = label_sentences(x_train, 'Train')
    x_test = label_sentences(x_test, 'Test')
    all_data = x_train + x_test
    return x_train, x_test, y_train, y_test, all_data

In [17]:
def read_dataset_df(df):
    dataset = df
    x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.outcome, random_state=0, test_size=0.3)
    x_train = label_sentences(x_train, 'Train')
    x_test = label_sentences(x_test, 'Test')
    all_data = x_train + x_test
    return x_train, x_test, y_train, y_test, all_data

In [4]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the LabeledSentence method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the review.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.LabeledSentence(v.split(), [label]))
    return labeled

In [5]:
def get_vectors(doc2vec_model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = doc2vec_model.docvecs[prefix]
    return vectors

In [25]:
def train_doc2vec(corpus):
    logging.info("Building Doc2Vec vocabulary")
    d2v = doc2vec.Doc2Vec(min_count=1,  # Ignores all words with total frequency lower than this
                          window=10,  # The maximum distance between the current and predicted word within a sentence
                          vector_size=300,  # Dimensionality of the generated feature vectors
                          workers=5,  # Number of worker threads to train the model
                          alpha=0.025,  # The initial learning rate
                          min_alpha=0.00025,  # Learning rate will linearly drop to min_alpha as training progresses
                          dm=0, # dm defines the training algorithm. If dm=1 means ‘distributed memory’ (PV-DM)
                                 # and dm =0 means ‘distributed bag of words’ (PV-DBOW)
                          dbow_words  = 1,
                          dm_mean = 1)  
    d2v.build_vocab(corpus)

    logging.info("Training Doc2Vec model")
    # 10 epochs take around 10 minutes on my machine (i7), if you have more time/computational power make it 20
    for epoch in range(10):
        logging.info('Training iteration #{0}'.format(epoch))
        d2v.train(corpus, total_examples=d2v.corpus_count, epochs=d2v.iter)
        # shuffle the corpus
        random.shuffle(corpus)
        # decrease the learning rate
        d2v.alpha -= 0.0002
        # fix the learning rate, no decay
        d2v.min_alpha = d2v.alpha

    logging.info("Saving trained Doc2Vec model")
    d2v.save("d2v.model")
    return d2v

In [20]:
def train_classifier(d2v, training_vectors, training_labels):
    logging.info("Classifier training")
    train_vectors = get_vectors(d2v, len(training_vectors), 300, 'Train')
    model = LinearSVC(penalty='l2')
    model.fit(train_vectors, np.array(training_labels))
    training_predictions = model.predict(train_vectors)
    logging.info('Training predicted classes: {}'.format(np.unique(training_predictions)))
    logging.info('Training accuracy: {}'.format(accuracy_score(training_labels, training_predictions)))
    logging.info('Training F1 score: {}'.format(f1_score(training_labels, training_predictions, average='weighted')))
    return model

In [8]:
def test_classifier(d2v, classifier, testing_vectors, testing_labels):
    logging.info("Classifier testing")
    test_vectors = get_vectors(d2v, len(testing_vectors), 300, 'Test')
    testing_predictions = classifier.predict(test_vectors)
    logging.info('Testing predicted classes: {}'.format(np.unique(testing_predictions)))
    logging.info('Testing accuracy: {}'.format(accuracy_score(testing_labels, testing_predictions)))
    logging.info('Testing F1 score: {}'.format(f1_score(testing_labels, testing_predictions, average='weighted')))

In [13]:
import os
import pandas as pd
from collections import Counter

PATH = '..\\Emotion Flow GitHub\\data_all\\'

data = []

for root, dirs, files in os.walk(PATH):
    for i,name in enumerate(files):
        if name.endswith((".txt")):
            dir_split = root.replace('.','')
            dir_split = dir_split.split('\\')
            category = dir_split[3]
            outcome = dir_split[4]
            file = open(root+'\\'+name, 'r', encoding='utf-8',errors='ignore')
            text_data = file.read()
            temp = []
            temp.append(category)
            temp.append(text_data)
            temp.append(outcome)
            data.append(temp)
            
            #print(category,outcome,name)
file.close()

In [14]:
df = pd.DataFrame(data)
df.columns = ['category','data','outcome']
print(len(df))
#df['value']=df.outcome.str[7:8]
df['outcome']=df.outcome.str[0:7]
print(df.head())

1003
            category                                               data  \
0  Detective_Mystery  \n\nE-text prepared by Juliet Sutherland, Mary...   
1  Detective_Mystery  PINES***\n\n\nE-text prepared by Juliet Suther...   
2  Detective_Mystery  \n\n\n\nProduced by Juliet Sutherland, Mary Me...   
3  Detective_Mystery  \n\nE-text prepared by Juliet Sutherland, Mary...   
4  Detective_Mystery  \n\n\n\nProduced by Steven desJardins and PG D...   

   outcome  
0  failure  
1  failure  
2  failure  
3  failure  
4  failure  


In [26]:
if __name__ == "__main__":
    x_train, x_test, y_train, y_test, all_data = read_dataset_df(df)
    d2v_model = train_doc2vec(all_data)
    classifier = train_classifier(d2v_model, x_train, y_train)
    test_classifier(d2v_model, classifier, x_test, y_test)

C:\Users\Prime\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  # Remove the CWD from sys.path while we load stuff.
2018-11-20 18:02:40,376 : INFO : Building Doc2Vec vocabulary
2018-11-20 18:02:40,376 : INFO : collecting all words and their counts
2018-11-20 18:02:40,376 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-20 18:02:42,922 : INFO : collected 511920 word types and 1003 unique tags from a corpus of 1003 examples and 13654253 words
2018-11-20 18:02:42,922 : INFO : Loading a fresh vocabulary
2018-11-20 18:02:44,075 : INFO : min_count=1 retains 511920 unique words (100% of original 511920, drops 0)
2018-11-20 18:02:44,076 : INFO : min_count=1 leaves 13654253 word corpus (100% of original 13654253, drops 0)
2018-11-20 18:02:45,205 : INFO : deleting the raw counts dictionary of 511920 items
2018

2018-11-20 18:03:51,486 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-20 18:03:51,539 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-20 18:03:51,546 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-20 18:03:51,547 : INFO : EPOCH - 1 : training on 13654253 raw words (8238048 effective words) took 59.2s, 139124 effective words/s
2018-11-20 18:03:52,703 : INFO : EPOCH 2 - PROGRESS: at 1.60% examples, 103426 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:03:53,754 : INFO : EPOCH 2 - PROGRESS: at 3.19% examples, 116183 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:03:54,867 : INFO : EPOCH 2 - PROGRESS: at 4.89% examples, 122009 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:03:56,047 : INFO : EPOCH 2 - PROGRESS: at 6.88% examples, 124165 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:03:57,202 : INFO : EPOCH 2 - PROGRESS: at 8.67% examples, 126220 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:03:58,249 : 

2018-11-20 18:05:01,566 : INFO : EPOCH 3 - PROGRESS: at 9.67% examples, 125625 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:05:02,650 : INFO : EPOCH 3 - PROGRESS: at 10.97% examples, 122141 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:05:03,696 : INFO : EPOCH 3 - PROGRESS: at 12.16% examples, 120218 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:05:04,736 : INFO : EPOCH 3 - PROGRESS: at 13.56% examples, 119489 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:05:05,770 : INFO : EPOCH 3 - PROGRESS: at 15.15% examples, 120512 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:05:06,829 : INFO : EPOCH 3 - PROGRESS: at 16.65% examples, 120333 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:05:07,836 : INFO : EPOCH 3 - PROGRESS: at 18.25% examples, 122544 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:05:08,949 : INFO : EPOCH 3 - PROGRESS: at 19.84% examples, 121714 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:05:09,944 : INFO : EPOCH 3 - PROGRESS: at 21.73% examples, 123844 words/s, in_qsi

2018-11-20 18:06:13,353 : INFO : EPOCH 4 - PROGRESS: at 18.44% examples, 124185 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:14,376 : INFO : EPOCH 4 - PROGRESS: at 20.34% examples, 125835 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:06:15,466 : INFO : EPOCH 4 - PROGRESS: at 22.03% examples, 125962 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:16,502 : INFO : EPOCH 4 - PROGRESS: at 23.73% examples, 127286 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:17,564 : INFO : EPOCH 4 - PROGRESS: at 25.52% examples, 128091 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:18,567 : INFO : EPOCH 4 - PROGRESS: at 27.22% examples, 127777 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:19,592 : INFO : EPOCH 4 - PROGRESS: at 28.91% examples, 128609 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:20,688 : INFO : EPOCH 4 - PROGRESS: at 30.71% examples, 129180 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:06:21,708 : INFO : EPOCH 4 - PROGRESS: at 32.60% examples, 129471 words/s, in_qsi

2018-11-20 18:07:26,949 : INFO : EPOCH 5 - PROGRESS: at 35.69% examples, 125861 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:07:27,966 : INFO : EPOCH 5 - PROGRESS: at 37.59% examples, 126590 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:07:28,996 : INFO : EPOCH 5 - PROGRESS: at 38.98% examples, 126416 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:07:29,998 : INFO : EPOCH 5 - PROGRESS: at 40.68% examples, 126929 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:07:31,014 : INFO : EPOCH 5 - PROGRESS: at 42.57% examples, 127245 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:07:32,119 : INFO : EPOCH 5 - PROGRESS: at 44.67% examples, 127790 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:07:33,193 : INFO : EPOCH 5 - PROGRESS: at 46.56% examples, 128211 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:07:34,227 : INFO : EPOCH 5 - PROGRESS: at 48.06% examples, 128293 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:07:35,244 : INFO : EPOCH 5 - PROGRESS: at 49.45% examples, 128273 words/s, in_

...

2018-11-20 18:48:30,564 : INFO : training on a 68271265 raw words (41190181 effective words) took 285.2s, 144403 effective words/s
2018-11-20 18:48:30,564 : INFO : Training iteration #9
2018-11-20 18:48:30,564 : INFO : training model with 5 workers on 511920 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2018-11-20 18:48:31,629 : INFO : EPOCH 1 - PROGRESS: at 1.50% examples, 125683 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:48:32,629 : INFO : EPOCH 1 - PROGRESS: at 3.29% examples, 139520 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:48:33,664 : INFO : EPOCH 1 - PROGRESS: at 5.28% examples, 139042 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:48:34,669 : INFO : EPOCH 1 - PROGRESS: at 6.88% examples, 141341 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:48:35,686 : INFO : EPOCH 1 - PROGRESS: at 8.57% examples, 141267 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:48:36,722 : INFO : EPOCH 1 - PROGRESS: at 10.37% examples, 142115 words/s, in_qsize 9

2018-11-20 18:49:40,005 : INFO : EPOCH 2 - PROGRESS: at 21.64% examples, 143619 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:41,024 : INFO : EPOCH 2 - PROGRESS: at 23.63% examples, 144047 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:42,135 : INFO : EPOCH 2 - PROGRESS: at 25.42% examples, 143902 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:43,221 : INFO : EPOCH 2 - PROGRESS: at 27.62% examples, 144440 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:44,269 : INFO : EPOCH 2 - PROGRESS: at 29.41% examples, 144335 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:45,293 : INFO : EPOCH 2 - PROGRESS: at 31.70% examples, 144746 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:46,318 : INFO : EPOCH 2 - PROGRESS: at 33.70% examples, 145093 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:47,324 : INFO : EPOCH 2 - PROGRESS: at 35.39% examples, 145048 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:49:48,451 : INFO : EPOCH 2 - PROGRESS: at 37.19% examples, 144886 words/s, in_qsiz

2018-11-20 18:50:52,089 : INFO : EPOCH 3 - PROGRESS: at 49.25% examples, 144037 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:50:53,111 : INFO : EPOCH 3 - PROGRESS: at 51.15% examples, 144283 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:50:54,137 : INFO : EPOCH 3 - PROGRESS: at 52.94% examples, 144169 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:50:55,181 : INFO : EPOCH 3 - PROGRESS: at 54.64% examples, 144262 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:50:56,203 : INFO : EPOCH 3 - PROGRESS: at 56.63% examples, 144199 words/s, in_qsize 10, out_qsize 1
2018-11-20 18:50:57,228 : INFO : EPOCH 3 - PROGRESS: at 58.52% examples, 144356 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:50:58,294 : INFO : EPOCH 3 - PROGRESS: at 60.22% examples, 144121 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:50:59,331 : INFO : EPOCH 3 - PROGRESS: at 62.11% examples, 144517 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:51:00,429 : INFO : EPOCH 3 - PROGRESS: at 63.91% examples, 144245 words/s, in_

2018-11-20 18:52:03,713 : INFO : EPOCH 4 - PROGRESS: at 76.37% examples, 146272 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:52:04,792 : INFO : EPOCH 4 - PROGRESS: at 78.07% examples, 145911 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:52:05,813 : INFO : EPOCH 4 - PROGRESS: at 79.86% examples, 146045 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:52:06,832 : INFO : EPOCH 4 - PROGRESS: at 81.56% examples, 145939 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:52:07,961 : INFO : EPOCH 4 - PROGRESS: at 83.65% examples, 146084 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:52:08,965 : INFO : EPOCH 4 - PROGRESS: at 85.64% examples, 146096 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:52:09,970 : INFO : EPOCH 4 - PROGRESS: at 87.34% examples, 146136 words/s, in_qsize 10, out_qsize 0
2018-11-20 18:52:10,970 : INFO : EPOCH 4 - PROGRESS: at 89.13% examples, 146220 words/s, in_qsize 9, out_qsize 0
2018-11-20 18:52:12,038 : INFO : EPOCH 4 - PROGRESS: at 90.73% examples, 145967 words/s, in_

2018-11-20 18:53:13,178 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-20 18:53:13,203 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-20 18:53:13,237 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-20 18:53:13,237 : INFO : EPOCH - 5 : training on 13654253 raw words (8237712 effective words) took 56.3s, 146276 effective words/s
2018-11-20 18:53:13,237 : INFO : training on a 68271265 raw words (41190077 effective words) took 282.7s, 145718 effective words/s
2018-11-20 18:53:13,237 : INFO : Saving trained Doc2Vec model
2018-11-20 18:53:13,237 : INFO : saving Doc2Vec object under d2v.model, separately None
2018-11-20 18:53:13,237 : INFO : storing np array 'syn1neg' to d2v.model.trainables.syn1neg.npy
2018-11-20 18:53:13,751 : INFO : storing np array 'vectors' to d2v.model.wv.vectors.npy
2018-11-20 18:53:15,511 : INFO : saved d2v.model
2018-11-20 18:53:15,965 : INFO : Classifier training
2018-11-20 18:53:16,1